In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [2]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [3]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [4]:
from pathlib import Path
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    text = d.get("reviewText","")
    rating = d.get("overall","")
    text_len = len(text.split(" "))
    category = path.name.strip(".json.gz")
    df[i] = category,rating,text,text_len
    i += 1
    if i == 6739590:
        break
  return pd.DataFrame.from_dict(df, columns=["category", "overall", "reviewText", "sentenceLength",], orient='index')

In [5]:
df = getDF(Path('../data/Books_5/Books_5.json.gz'))

In [6]:
df.shape

(6739590, 4)

In [7]:
df.head()

,category,overall,reviewText,sentenceLength
0,Books_5,5.0,"The King, the Mice and the Cheese by Nancy Gurney is an excellent children's book. It is one that I well remember from my own childhood and purchased for my daughter who loves it.\n\nIt is about a king who has trouble with rude mice eating his cheese. He consults his wise men and they suggest cats to chase away the mice. The cats become a nuisance, so the wise men recommend the king bring in dogs to chase the cats away. The cycle goes on until the mice are finally brought back to chase away the elephants, brought in to chase away the lions that'd chased away the dogs.\n\nThe story ends i...",164
1,Books_5,5.0,The kids loved it!,4
2,Books_5,5.0,My students (3 & 4 year olds) loved this book! Definitely recommend it to other teachers.,17
3,Books_5,5.0,LOVE IT,2
4,Books_5,5.0,Great!,1


In [8]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [9]:
sample_df.shape

(336980, 5)

In [10]:
del df
gc.collect()

375

In [11]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [12]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [13]:
learn.load("gpt2-finetuned-amazon-reviews-sample-1cycle-Books_5").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [14]:
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2500
BINS = list(range(10,351))

In [15]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [16]:
generated_df.shape

(4370, 4)

In [17]:
generated_df.head()

,category,rating,text,label
0,Books_5,5.0,I'm hooked on this writer and will be reading more of her work.\n\nThe premise of,CG
1,Books_5,5.0,Good book and exactly as described. The characters were well developed and believable. The writing was just as,CG
2,Books_5,5.0,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who",CG
3,Books_5,5.0,Absolutely loved every word! We have the book on Kindle and it's also a must read!,CG
4,Books_5,5.0,"I loved it! It was a real, believable, and engrossing read. I would recommend",CG


In [18]:
def post_process(str, k=3):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [19]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [20]:
generated_df.head()

,category,rating,text,label,text_
0,Books_5,5.0,I'm hooked on this writer and will be reading more of her work.\n\nThe premise of,CG,I'm hooked on this writer and will be reading more of her work.
1,Books_5,5.0,Good book and exactly as described. The characters were well developed and believable. The writing was just as,CG,Good book and exactly as described. The characters were well developed and believable. The writing was just as
2,Books_5,5.0,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who",CG,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who"
3,Books_5,5.0,Absolutely loved every word! We have the book on Kindle and it's also a must read!,CG,Absolutely loved every word! We have the book on Kindle and it's also a must read!
4,Books_5,5.0,"I loved it! It was a real, believable, and engrossing read. I would recommend",CG,"I loved it! It was a real, believable, and engrossing read."


In [21]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [22]:
generated_df.head()

,category,rating,label,text_
0,Books_5,5.0,CG,I'm hooked on this writer and will be reading more of her work.
1,Books_5,5.0,CG,Good book and exactly as described. The characters were well developed and believable. The writing was just as
2,Books_5,5.0,CG,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who"
3,Books_5,5.0,CG,Absolutely loved every word! We have the book on Kindle and it's also a must read!
4,Books_5,5.0,CG,"I loved it! It was a real, believable, and engrossing read."


In [23]:
generated_df.to_csv("Generated_Sentences_Books_5.csv", index=None)